!pip install BeautifulSoup4, lxml, mapbox, pandas, requests, IPython

In [ ]:
import json, config
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [ ]:
from mapbox import Geocoder
geocoder = Geocoder(access_token=config.api_key)

In [ ]:
url = "https://en.wikipedia.org/wiki/List_of_National_Hockey_League_arenas"

In [ ]:
page = urlopen(url)

In [ ]:
soup = BeautifulSoup(page, "lxml")

print(soup.prettify())

In [ ]:
first_table = soup.find('table', class_= 'wikitable sortable')

In [ ]:
A=[] #IMG
B=[] #Arena
C=[] #City
D=[] #Team
E=[] #Capacity
F=[] #Opened
G=[] #First Season
H=[] #State

In [ ]:
for row in first_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==8:
        A.append(cells[0].find('img')['src'].replace("120px", "250px"))
        B.append(cells[1].find(text=True))
        #this splits the city and state then puts them in their own arrays.
        location = cells[2].find(text=True).split(',')

        C.append(location[0])
        H.append(location[1].lstrip())
        D.append(cells[3].find(text=True))
        E.append(cells[4].find(text=True).rstrip('\n'))
        F.append(cells[5].find(text=True).rstrip('\n').rstrip('\u2020').replace(' ', ''))
        G.append(cells[6].find(text=True).rstrip('\n').replace('\u2013', '-'))

In [ ]:
import pandas as pd
df=pd.DataFrame(D,columns=['Team'])
df['Arena']=B
df['City']=C
df['State'] = H
df['Capacity']=E
df['opened']=F
df['firstSeason']=G
df['image']=A
df

In [ ]:
location = [];
for i in range(len(B)):
    searchTerm = B[i] + ", " + C[i]
    #print(searchTerm)
    response = geocoder.forward(searchTerm)
    collection = response.json()
    data = collection['features'][0]['geometry']['coordinates']
    location.append(data[::-1])

In [ ]:
response = geocoder.forward('Bell Centre,  Quebec')
collection = response.json()
data = collection['features'][0]['geometry']['coordinates']
#print(collection);

In [ ]:
final_data = []
for i in range(len(A)):
    temp={}
    temp['Image'] = "HTTPS:" + A[i]
    temp['Team'] = D[i]
    temp['Arena'] = B[i]
    temp['Location'] = C[i]
    temp['Capacity'] = E[i]
    temp['opened'] = F[i]
    temp['First Season'] = G[i]
    temp['Coordinates'] = location[i]
    final_data.append(temp)

In [ ]:
print(len(final_data))

In [ ]:
with open('NHL.json', 'w') as outfile:
    json.dump(final_data, outfile)

In [ ]:
import requests, IPython.display as Disp
url = final_data[14]['Image']
Disp.Image(requests.get(url).content)

In [ ]:
print(url)